In [9]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn import linear_model, preprocessing
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler

In [1]:
import pandas as pd
import numpy as np
from pandas import Series,DataFrame
from statistics import mean

In [2]:
def elastic_cv(X,y,k,c):
    coef = []
    r2_train = []
    r2_test = []
    r2 = []
    all_pred = []
    all_y = []
    kf = KFold(n_splits = k, shuffle = True)
    clf = ElasticNet(alpha=0.1,l1_ratio=0.9)
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        ss = StandardScaler()
        X_train = ss.fit_transform(X_train)
        X_test = ss.transform(X_test)
        clf.fit(X_train, y_train) 
        coef.append(clf.coef_)
        pred_test = clf.predict(X_test)
        r2_train.append(r2_score(y_train, clf.predict(X_train)))
        r2_test.append(r2_score(y_test, pred_test))
        all_pred.append(pred_test)
        all_y.append(y_test.values)
        
    coef_mean = []
    for j in range(c):
        coef_j = []
        for l in range(k):
            coef_j.append(coef[l][j])
        coef_mean.append(mean(coef_j))
    print("回帰変数", coef_mean)
    
    from scipy.stats import pearsonr
  
    a, b = pearsonr(np.ravel(all_pred), np.ravel(all_y))
    print("相関係数:", a)
    print("p値:", b)

    print('r2_test:',mean(r2_test)) #r2_test: 決定係数
    return coef_mean

In [3]:
import numpy as np
def mean_entropy(ppg):
    def H(x):
        return x*np.log2(x) if x!=0 else 0
    H = np.vectorize(H)
    return np.apply_along_axis(lambda x: -sum(H(x)) , 1, ppg).mean()

def accum_diff(ppg):
    accum = 0
    for i in range(len(ppg)-1):
        accum += np.linalg.norm(ppg[i]-ppg[i+1], ord=2)
    return accum/len(ppg)

def mean_max(ppg):
    return ppg.max(axis=1).mean()
def mean_std(ppg):
    return ppg.std(axis=1).mean()

def diff_from_nt(dic, ppg):
    mean_dic = {}
    for i in range(100):
            m = dic[f"UTT{i+1}"].mean(axis=0)
            mean_dic[f"UTT{i+1}"] = m

    mean_nt = np.array(list(mean_dic.values())[-10:]).mean(axis=0)
    dif_dec = {}
    for k, v in mean_dic.items():
        dif_dec[k] = np.linalg.norm(v-mean_nt, ord=2)
        
    return dif_dec
    
a = np.array([[0.5,0.5],[0.5,1],[1.5,1.5]])
mean_std(a)

0.08333333333333333

In [4]:
import pandas as pd

df = pd.read_csv("main_data_1.csv", index_col=0)
df["UTT"] = df["ID"].apply(lambda x: f"UTT{x-100}")

In [5]:
df = pd.read_csv("csvs/ent.csv", index_col=0)

In [6]:
import os
import pickle
import pandas as pd

df = pd.read_csv("csvs/ent_diff_max_std.csv", index_col=0)
ppgs = os.listdir("ppgs")
for p in ppgs:
    with open(f"ppgs/{p}", "rb") as f:
        dic = pickle.load(f)
    #各話者のPPGの平均
    mean_dic = {}
    for i in range(100):
            m = dic[f"UTT{i+1}"].mean(axis=0)
            mean_dic[f"UTT{i+1}"] = m

    mean_nt = np.array(list(mean_dic.values())[-10:]).mean(axis=0)
    ##
    entropy = []
    diff_nt = []
    for k,v in dic.items():
        # entropy.append([k, mean_entropy(v)])
        diff_nt.append([k,np.linalg.norm(v-mean_nt, ord=2)])
    print(p)
    # ent_df = pd.DataFrame(entropy,columns=["UTT", f"ent_{p.split('_')[-1]}"])
    # df = pd.merge(df, ent_df, on="UTT", how="left")

    std_df = pd.DataFrame(diff_nt,columns=["UTT", f"diff_nt_{p.split('_')[-1]}"])
    df = pd.merge(df, std_df, on="UTT", how="left")


df.to_csv("csvs/ent_diff_max_std_nt.csv")

ppg_20
ppg_70
ppg_300
ppg_800
ppg_400
ppg_10
ppg_1800
ppg_60
ppg_1600
ppg_500
ppg_80
ppg_700
ppg_200
ppg_5
ppg_100
ppg_30
ppg_40
ppg_1000
ppg_90
ppg_2000
ppg_900
ppg_1200
ppg_1400
ppg_600
ppg_50


In [10]:
import pandas as pd

df = pd.read_csv("csvs/ent_diff_max_std_nt.csv", index_col=0)
X = df.iloc[:,108:]
y = df.loc[:,"FR"]
X

,diff_nt_20,diff_nt_70,diff_nt_300,diff_nt_800,diff_nt_400,diff_nt_10,diff_nt_1800,diff_nt_60,diff_nt_1600,diff_nt_500,...,diff_nt_30,diff_nt_40,diff_nt_1000,diff_nt_90,diff_nt_2000,diff_nt_900,diff_nt_1200,diff_nt_1400,diff_nt_600,diff_nt_50
0,17.048720,16.614834,14.923130,10.716284,14.917736,17.875485,10.171103,16.730636,10.176508,12.748669,...,16.842380,16.788737,10.713602,16.581946,10.121636,10.714766,10.711157,10.709908,12.746205,16.742012
1,23.375347,22.860413,21.646744,17.311070,21.635554,23.864050,10.147015,23.137361,13.480021,17.316542,...,23.373741,23.251102,17.307690,22.805329,9.807681,17.309100,17.305671,17.304315,17.313991,23.170277
2,24.805788,24.842979,24.475774,17.058201,24.464019,25.243994,14.629810,25.036431,16.663363,23.065175,...,25.259637,25.170077,17.051201,24.795626,13.605295,17.054383,17.047762,17.045143,23.061974,25.066800
3,24.865799,23.052467,21.132839,18.035572,21.118536,25.485068,17.579511,24.423734,17.583188,18.777926,...,24.671340,24.560675,18.031584,23.002492,17.572872,18.033240,18.028479,18.026316,18.773396,24.451506
4,21.453004,20.492198,18.550639,13.306145,18.542486,22.308480,9.106429,21.077281,10.301890,13.309167,...,21.264978,21.182687,13.304477,20.445119,8.166344,13.305914,13.302493,13.301450,13.306202,21.111641
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,16.783458,13.768956,10.485987,9.086177,10.470870,17.808287,9.074699,15.490873,9.074868,9.132358,...,15.619994,15.580930,9.084828,13.744478,8.435167,9.085316,9.083706,9.082553,9.130954,15.499970
96,19.572876,18.238328,16.672173,11.751366,16.664621,20.244006,11.362328,19.034224,11.368956,12.357400,...,19.196588,19.122655,11.748819,18.206581,11.161630,11.750109,11.746726,11.744879,12.354691,19.061214
97,12.263033,9.079549,6.810301,5.529544,6.794337,13.805647,5.490476,10.637699,5.493656,5.620167,...,10.752252,10.730140,5.524694,9.038520,4.398747,5.526982,5.522041,5.519788,5.610314,10.662646
98,20.038740,15.724970,12.623940,9.982195,12.615836,21.220161,9.213300,18.361351,9.215222,10.980908,...,18.521673,18.433565,9.972735,15.696263,9.227510,9.978852,9.968320,9.961035,10.977237,18.400621


In [11]:
coef = elastic_cv(X,y,5,25)

回帰変数 [0.0, -0.15022674011712678, -0.44278408237619654, -0.0016982967032271247, -0.4074385423193301, 0.0, 0.0, 0.0, 0.0, -0.006039559768132347, -0.17045378098180164, 0.0, -0.03165705358370685, 0.0, -0.161920315317625, -0.016538588322854566, 0.0, 0.0, -0.16122715124221376, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
相関係数: 0.7392536200035933
p値: 1.6046156220130858e-18
r2_test: 0.5092037706352194


In [12]:
import pandas as pd

df = pd.read_csv("csvs/ent_diff_max_std_nt.csv", index_col=0)
# X = df.loc[:,['ent_1800','ent_2000','diff_400','diff_2000', 'diff_1400']]
X = df.loc[:, ["ent_2000", "diff_2000","diff_nt_2000"]]
# X = df.loc[:, ["diff_2000"]]
y = df.loc[:,"FR"]

In [13]:
coef = elastic_cv(X,y,5,3)

回帰変数 [-0.4887491756257861, 1.387128897008032, -0.3366560101343036]
相関係数: 0.8613259217264099
p値: 1.4048217162155539e-30
r2_test: 0.684077501432759


In [14]:
[X.columns[i] for i in range(len(coef)) if coef[i]]

['ent_2000', 'diff_2000', 'diff_nt_2000']

In [15]:
df["diff_nt_500"].sort_values()[:20]

49     5.404507
97     5.620167
99     6.872865
92     7.074845
45     7.741858
52     7.981843
30     9.010718
61     9.072211
38     9.096300
51     9.129049
95     9.132358
68     9.181103
47     9.198313
56     9.470118
24     9.531856
21     9.636181
70     9.859428
41     9.893087
23    10.103048
34    10.204198
Name: diff_nt_500, dtype: float64

In [16]:
non = df.iloc[:90]
nt = df.iloc[90:]

In [17]:
features=['ent_20', 'ent_70', 'ent_300', 'ent_800', 'ent_400', 'ent_10',
       'ent_1800', 'ent_60', 'ent_1600', 'ent_500', 'ent_80', 'ent_700',
       'ent_200', 'ent_5', 'ent_100', 'ent_30', 'ent_40', 'ent_1000', 'ent_90',
       'ent_2000', 'ent_900', 'ent_1200', 'ent_1400', 'ent_600', 'ent_50',
       'diff_20', 'diff_70', 'diff_300', 'diff_800', 'diff_400', 'diff_10',
       'diff_1800', 'diff_60', 'diff_1600', 'diff_500', 'diff_80', 'diff_700',
       'diff_200', 'diff_5', 'diff_100', 'diff_30', 'diff_40', 'diff_1000',
       'diff_90', 'diff_2000', 'diff_900', 'diff_1200', 'diff_1400',
       'diff_600', 'diff_50']

In [18]:
comp = pd.concat([pd.DataFrame(non[features].mean()),pd.DataFrame(nt[features].mean())], axis=1).set_axis(["non", "nt"],axis=1)
comp

,non,nt
ent_20,0.839246,0.746028
ent_70,1.119544,0.995922
ent_300,1.457140,1.287660
ent_800,1.805027,1.547617
ent_400,1.520413,1.346352
ent_10,0.678953,0.598691
ent_1800,2.144933,1.830410
ent_60,1.048717,0.931040
ent_1600,2.057952,1.766470
ent_500,1.647925,1.432281


In [19]:
with open(f"ppgs/ppg_2000", "rb") as f:
        dic = pickle.load(f)
mean_dic = {}
for i in range(100):
        m = dic[f"UTT{i+1}"].mean(axis=0)
        mean_dic[f"UTT{i+1}"] = m
mean_dic
        

{'UTT1': array([4.1186432e-03, 3.1502782e-03, 2.0311521e-03, ..., 7.9492165e-05,
        2.2716704e-03, 3.7733553e-06], dtype=float32),
 'UTT2': array([4.0710350e-03, 1.1529774e-02, 1.8761804e-02, ..., 1.2641671e-03,
        8.4374635e-04, 1.9130723e-05], dtype=float32),
 'UTT3': array([6.2955245e-03, 1.1008184e-02, 3.2487037e-03, ..., 2.2677447e-04,
        2.7349204e-04, 2.0043441e-05], dtype=float32),
 'UTT4': array([1.2921637e-02, 5.1825927e-03, 4.3514981e-03, ..., 1.5545454e-03,
        4.5337557e-04, 7.8286132e-07], dtype=float32),
 'UTT5': array([9.1635184e-03, 7.1544633e-03, 9.3300808e-03, ..., 1.1769549e-04,
        8.1766990e-04, 4.8483103e-06], dtype=float32),
 'UTT6': array([9.3786390e-03, 8.3568180e-03, 1.4069882e-02, ..., 2.4145102e-05,
        4.9329258e-04, 9.3139033e-06], dtype=float32),
 'UTT7': array([1.1624310e-02, 1.7268380e-02, 5.8958246e-03, ..., 1.8763112e-05,
        2.7806507e-04, 5.6359488e-05], dtype=float32),
 'UTT8': array([7.0045427e-03, 4.2710099e-03, 6.

In [20]:
mean_nt = np.array(list(mean_dic.values())[-10:]).mean(axis=0)
dif_dec = {}
for k, v in mean_dic.items():
    dif_dec[k] = np.linalg.norm(v-mean_nt, ord=2)
dif_dec

{'UTT1': 0.10414168,
 'UTT2': 0.08920294,
 'UTT3': 0.1594773,
 'UTT4': 0.11539289,
 'UTT5': 0.07163105,
 'UTT6': 0.10847793,
 'UTT7': 0.12229005,
 'UTT8': 0.07053613,
 'UTT9': 0.13243437,
 'UTT10': 0.0668263,
 'UTT11': 0.11709256,
 'UTT12': 0.06645237,
 'UTT13': 0.11661282,
 'UTT14': 0.08529442,
 'UTT15': 0.16096596,
 'UTT16': 0.0952731,
 'UTT17': 0.11487567,
 'UTT18': 0.15449099,
 'UTT19': 0.0930317,
 'UTT20': 0.06361807,
 'UTT21': 0.13962951,
 'UTT22': 0.06104036,
 'UTT23': 0.1250393,
 'UTT24': 0.06264034,
 'UTT25': 0.071024366,
 'UTT26': 0.06343362,
 'UTT27': 0.18696389,
 'UTT28': 0.114248276,
 'UTT29': 0.07477877,
 'UTT30': 0.07379867,
 'UTT31': 0.046817727,
 'UTT32': 0.14955612,
 'UTT33': 0.13286711,
 'UTT34': 0.1153446,
 'UTT35': 0.08696943,
 'UTT36': 0.10298464,
 'UTT37': 0.09478509,
 'UTT38': 0.06085025,
 'UTT39': 0.07488333,
 'UTT40': 0.057545085,
 'UTT41': 0.07322607,
 'UTT42': 0.063760966,
 'UTT43': 0.047104668,
 'UTT44': 0.09291785,
 'UTT45': 0.07205538,
 'UTT46': 0.0669331